<a href="https://colab.research.google.com/github/arturomolin/Tarea1/blob/main/Ejercicio_27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.linalg import qr, solve

# Tomamos al azar valores de epsilon cercanos a sqrt(eps_m) y eps_m, pero no iguales
epsilons = [1e-7, 5e-8, 1e-15]
b = np.array([1, 0, 0, 0], dtype=float)

# Como haremos distintos epsilons, necesitamos que la matriz A "mute" de acuerdo a cada epsilon
def construir_A(eps):
    return np.array([
        [1, 1, 1],
        [eps, 0, 0],
        [0, eps, 0],
        [0, 0, eps]
    ], dtype=float)

#Y planteamos los métodos
# Ecuaciones normales
def ecuaciones_normales(A, b):
    AtA = A.T @ A
    Atb = A.T @ b
    return solve(AtA, Atb)

# Householder ---
def householder(A, b):
    Q, R = qr(A, mode='economic')
    return solve(R, Q.T @ b)

# Givens (misma función en scipy para QR)
def givens(A, b):
    Q, R = qr(A, mode='economic')
    return solve(R, Q.T @ b)

# Gram-Schmidt
def gram_schmidt(A, b):
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    for j in range(n):
        v = A[:, j]
        for i in range(j):
            R[i, j] = np.dot(Q[:, i], A[:, j])
            v -= R[i, j] * Q[:, i]
        R[j, j] = np.linalg.norm(v)
        Q[:, j] = v / R[j, j]
    return solve(R, Q.T @ b)

methods = {
    "Ecuaciones Normales": ecuaciones_normales,
    "Householder": householder,
    "Givens": givens,
    "Gram-Schmidt": gram_schmidt
}

# Y resolvemos
for eps in epsilons:
    A = construir_A(eps)
    print(f"\n======= ε = {eps:.1e} =======")
    for name, method in methods.items():
        try:
            x = method(A, b)
            residuo = np.linalg.norm(A @ x - b)
            print(f"{name:25s} → x = {x}, residuo = {residuo:.2e}")
        except Exception as e:
            print(f"{name:25s} → ERROR: {e}")



======= ε = 1.0e-07 =======
Ecuaciones Normales       → x = [0.33333333 0.33333333 0.33333333], residuo = 5.77e-08
Householder               → x = [0.33333333 0.33333333 0.33333333], residuo = 5.77e-08
Givens                    → x = [0.33333333 0.33333333 0.33333333], residuo = 5.77e-08
Gram-Schmidt              → x = [0.34866916 0.32566542 0.32566542], residuo = 6.51e-01

======= ε = 5.0e-08 =======
Ecuaciones Normales       → x = [0.33333333 0.33333333 0.33333333], residuo = 2.89e-08
Householder               → x = [0.33333333 0.33333333 0.33333333], residuo = 2.89e-08
Givens                    → x = [0.33333333 0.33333333 0.33333333], residuo = 2.89e-08
Gram-Schmidt              → x = [0.40788105 0.29605947 0.29605947], residuo = 5.92e-01

======= ε = 1.0e-15 =======
Ecuaciones Normales       → ERROR: Matrix is singular.
Householder               → x = [0.33333333 0.33333333 0.33333333], residuo = 5.77e-16
Givens                    → x = [0.33333333 0.33333333 0.33333333], residuo